In [1]:
# May need to install these....
# uncomment if unsure
# !pip install wget
# !pip install dtale

In [2]:
import pandas as pd
import numpy as np
import os as os

import dtale # data Frame visualization
import wget as wget

# Download the data

Download Current employment statistics from [US Bureau of Labor Statics website](https://www.bls.gov/)


Data Dictionary
???



In [3]:
# Check if file has ben downloaded if not download the file
# This could take a while first time downloading the files (over 5 mins)
# Does this qualify as web scraping? :-)
lnFileName = 'resources/ALLlnSeries.txt'
lnSeriesFileName = 'resources/lnSeries.txt'
lnIndustryFileName = 'resources/lnIndustries.txt'
if not os.path.exists(lnFileName) :
    url = "https://download.bls.gov/pub/time.series/ln/ln.data.1.AllData"
    wget.download(url,lnFileName)
    
    url = "https://download.bls.gov/pub/time.series/ln/ln.series"    
    wget.download(url,lnSeriesFileName)
    
    url = "https://download.bls.gov/pub/time.series/ln/ln.indy"    
    wget.download(url,lnIndustryFileName)  

allLN =  pd.read_table(lnFileName, sep='\t', header=0,names=['series_id', 'year', 'period', 'value', 'footnote_codes'])

LNSeriesDF =  pd.read_table(lnSeriesFileName, sep='\t', header=0,
                            names=['series_id', 'lfst_code', 'periodicity_code', 'series_title','absn_code', 'activity_code', 'ages_code', 'cert_code', 'class_code',
                                   'duration_code', 'education_code', 'entr_code', 'expr_code',
                                   'hheader_code', 'hour_code', 'indy_code', 'jdes_code', 'look_code',
                                   'mari_code', 'mjhs_code', 'occupation_code', 'orig_code', 'pcts_code',
                                   'race_code', 'rjnw_code', 'rnlf_code', 'rwns_code', 'seek_code',
                                   'sexs_code', 'tdat_code', 'vets_code', 'wkst_code', 'born_code',
                                   'chld_code', 'disa_code', 'seasonal', 'footnote_codes', 'begin_year',
                                   'begin_period', 'end_year', 'end_period'],
                            dtype = { 'value':float,'indy_code':np.str_,  'ages_code':np.str_,'education_code':np.str_,'race_code':np.str_}) 


industriesDF =  pd.read_table(lnIndustryFileName, sep='\t', header=0,dtype={'indy_code':np.str_})    

#Clean up spaces on the series_id
allLN['series_id'] = allLN['series_id'].str.strip()
LNSeriesDF['series_id'] = LNSeriesDF['series_id'].str.strip()
#LNSeriesDF.head()

/Users/harinirao/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
# Only interested in 2015-2020
# Monthly data (not quarterly)
# CES Seasonally Adjusted
condition = allLN['year'].isin([2015,2016,2017,2018,2019,2020])  & allLN['period'].str.startswith('M')
LNS2020DF = allLN[condition]
LNS2020DF.shape



(1033192, 5)

In [5]:
# Get month from period and dop period
# month 13 Annual Average      
# https://download.bls.gov/pub/time.series/ce/ce.period
LNS2020DF['Month'] = LNS2020DF['period'].str.replace('M','').astype(int)
LNS2020DF.drop('period',1,inplace=True)
LNS2020DF.shape

/Users/harinirao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/harinirao/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(1033192, 5)

In [6]:
# Merge with Series DF
LNS2020DF = pd.merge(LNS2020DF,LNSeriesDF,on='series_id',how='left')
LNS2020DF.shape


(1033192, 45)

In [7]:
LNS2020DF = pd.merge(LNS2020DF,industriesDF,on='indy_code',how='left')
LNS2020DF.shape

(1033192, 46)

In [8]:
LNS2020DF.drop(['jdes_code','absn_code','cert_code','rjnw_code', 'begin_year',
                'begin_period', 'end_year', 'end_period', 'hheader_code', 'hour_code', 'indy_code', 'look_code',
       'mari_code', 'mjhs_code', 'occupation_code', 'orig_code', 'pcts_code',
       'race_code', 'rnlf_code', 'rwns_code', 'seek_code', 'sexs_code',
       'tdat_code', 'vets_code', 'wkst_code', 'born_code', 'chld_code',
       'disa_code', 'seasonal', 'footnote_codes_x','footnote_codes_y'],1,inplace=True)

LNS2020DF.shape

(1033192, 15)

In [9]:

Series_of_interest = [
    'LNS11000000', # Labor Force 
    'LNS11000001', # Labor Force Level Men
    'LNS11000002', # Labor Force Level Women
#     'LNS11000003', # Race White
#     'LNS11000006', # Black or African American
#     'LNS11032183', # Asian
#     'LNS11000009', # Hispanic or Latino
#     'LNS11000012', # 16-19
#     'LNS11000024', # 20 years & older
#     'LNS11300036', # 20-24
#     'LNS11300089', # 25-34
#     'LNS11300091', # 35-44
#     'LNS11300093', # 45-54
#     'LNS11324230', # 55 and over
    
#     'LNS11327659', # Less than HS
#     'LNS11327660', # HS No college
#     'LNS11327662', # Bachelor's degree and higher
#     'LNS11327689', #Some College or Associate Degree
#     'LNS12000000', # Employment Level
    
    'LNS13000000', # Unemployment Level
    'LNS13000001', # Unemployment Level Men
    'LNS13000002', # Unemployment Level Women
    'LNS13000003', # Unemployment Level White
    'LNS13000006', # Unemployment Level Black or African American
    'LNS13000009', # Unemployment Level Hispanic or Latino
    'LNS13032183', # Unemployment Level Asian
    'LNS13000012', # Unemployment Level 16-19
    'LNS13000036', # Unemployment Level 20-24
    'LNS13000089', # Unemployment Level 25-34
    'LNS13000091', # Unemployment Level 35-44
    'LNS13000093', # Unemployment Level 45-54
    'LNS13024230', # Unemployment Level 55 and over
    'LNS13027659', # Unemployment Level Less than HS
    'LNS13027660', # Unemployment Level HS No college
    'LNS13027662', # Unemployment Level Bachelor's degree and higher
    'LNS13027689', # Unemployment Level Some College or Associate Degree 
     
    'LNS13008396', # Number Unemployed for Less than 5 Weeks
    'LNS13008756', # Number Unemployed for 5-14 Weeks
    'LNS13008516', # Number Unemployed for 15 Weeks & over
    'LNS13008876', # Number Unemployed for 15-26 Weeks
    'LNS13008636', # Number Unemployed for 27 Weeks & over
#    'LNS13008275', # Average Weeks Unemployed
#    'LNS13008276', # Median Weeks Unemployed
    
    'LNS13023569', # Unemployment Level - New Entrants
    'LNS13023621', # Unemployment Level - Job Losers
    'LNS13023653', # Unemployment Level - Job Losers on Layoff
    'LNS13023705', # Unemployment Level - Job Leavers
    'LNS13025699', # Unemployment Level - Job Losers Not on Layoff
    'LNS13026637', # Unemployment Level - Persons who Completed Temporary Jobs
    'LNS13026638', # Unemployment Level - Permanent Job Losers

]    

condition =LNS2020DF['series_id'].isin(Series_of_interest) 
LNS2020DF = LNS2020DF[condition]
LNS2020DF.shape

(2080, 15)

In [10]:
# d = dtale.show(LNS2020DF, ignore_duplicate=True)
# d.open_browser()

In [11]:
LNS2020DF['value'] = pd.to_numeric(LNS2020DF['value'])

In [12]:
LNS2020DF.to_pickle("resources/LNS2020.pkl",compression='gzip')

In [13]:
# to load Picke file use the following once pickle has been generated
LNS2020DF_2=pd.read_pickle('resources/LNS2020.pkl',compression='gzip')